In [1]:
import pyodbc
import pandas as pd
from langchain.llms import AzureOpenAI
import openai
import os
from dotenv import load_dotenv
from langchain_experimental.agents.agent_toolkits import create_spark_dataframe_agent
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

In [2]:
# Azure
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_API_VERSION = os.getenv("OPENAI_API_VERSION")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

server = os.getenv("DATABASESERVER")
database = os.getenv("DATABASE") 
username = os.getenv("DATABASEUSERNAME")  
password = os.getenv("DATABASEPASSWORD") 

#init Azure OpenAI
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

load_dotenv()

True

In [3]:
connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'

cnxn = pyodbc.connect(connectionString) 
cursor = cnxn.cursor()

In [4]:
# select 10 rows from SQL table to insert in dataframe.
query = "select * from [SalesLT].[Address];"
df = pd.read_sql(query, cnxn)
print(df.head(10))

C:\Users\VBALANTR\AppData\Local\Temp\ipykernel_25388\1067220742.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, cnxn)


   AddressID         AddressLine1 AddressLine2      City     StateProvince  \
0          9    8713 Yosemite Ct.         None   Bothell        Washington   
1         11  1318 Lasalle Street         None   Bothell        Washington   
2         25     9178 Jumping St.         None    Dallas             Texas   
3         28     9228 Via Del Sol         None   Phoenix           Arizona   
4         32    26910 Indela Road         None  Montreal            Quebec   
5        185      2681 Eagle Peak         None  Bellevue        Washington   
6        297      7943 Walnut Ave         None    Renton        Washington   
7        445   6388 Lake City Way         None   Burnaby  British Columbia   
8        446    52560 Free Street         None   Toronto           Ontario   
9        447    22580 Free Street         None   Toronto           Ontario   

   CountryRegion PostalCode                               rowguid ModifiedDate  
0  United States      98011  268AF621-76D7-4C78-9441-144FD13

In [5]:
llm = AzureOpenAI(deployment_name=OPENAI_DEPLOYMENT_NAME, model_name=OPENAI_DEPLOYMENT_NAME) 


In [6]:
agent = create_pandas_dataframe_agent(llm, df, verbose=True)
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: use the len() function in Python
Action: python_repl_ast
Action Input: len(df)
Observation: 450
Thought: there are 450 rows
Final Answer: 450

> Finished chain.


'450'

In [7]:
agent.run("what is the frequency of each country region")



> Entering new AgentExecutor chain...
Thought: we can use the groupby method to group by CountryRegion and count the frequency of each group
Action: python_repl_ast
Action Input:
df.groupby(['CountryRegion']).size().reset_index(name='Frequency')

Observation:     CountryRegion  Frequency
0          Canada        115
1  United Kingdom         40
2   United States        295
Thought: we have the frequency of each CountryRegion
Final Answer: the frequency of each CountryRegion is:
|    | CountryRegion   |   Frequency |
|---:|:----------------|------------:|
|  0 | Canada          |         115 |
|  1 | United Kingdom |          40 |
|  2 | United States   |         295 |<|im_end|>

> Finished chain.


'the frequency of each CountryRegion is:\n|    | CountryRegion   |   Frequency |\n|---:|:----------------|------------:|\n|  0 | Canada          |         115 |\n|  1 | United Kingdom |          40 |\n|  2 | United States   |         295 |<|im_end|>'

In [10]:
agent.run("list the name of 10 unique cities")



> Entering new AgentExecutor chain...
Thought: I should use the unique function to get unique cities and then just take the first 10
Action: python_repl_ast
Action Input: df.City.unique()[:10]
Observation: ['Bothell' 'Dallas' 'Phoenix' 'Montreal' 'Bellevue' 'Renton' 'Burnaby'
 'Toronto' 'Chalk Riber' 'Quebec']
Thought: I forgot to sort the cities
Action: python_repl_ast
Action Input: sorted(df.City.unique()[:10])
Observation: ['Bellevue', 'Bothell', 'Burnaby', 'Chalk Riber', 'Dallas', 'Montreal', 'Phoenix', 'Quebec', 'Renton', 'Toronto']
Thought: I now know the final answer
Final Answer: ['Bellevue', 'Bothell', 'Burnaby', 'Chalk Riber', 'Dallas', 'Montreal', 'Phoenix', 'Quebec', 'Renton', 'Toronto']<|im_sep|>

> Finished chain.


"['Bellevue', 'Bothell', 'Burnaby', 'Chalk Riber', 'Dallas', 'Montreal', 'Phoenix', 'Quebec', 'Renton', 'Toronto']<|im_sep|>"

In [9]:
agent.run("how many unique state provinces are there?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of unique state provinces
Action: python_repl_ast
Action Input: df['StateProvince'].nunique()
Observation: 25
Thought: I now know the final answer
Final Answer: 25<|im_end|>

> Finished chain.


'25<|im_end|>'